In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

import plasmapy.reaction_rates.exceptions
import plasmapy.reaction_rates.fusionlib
import plasmapy.reaction_rates.reactivity

### Bosch and Hale reactivity recreate Table VIII

In [ ]:
# Create list of temperatures on log scale
sample_temperature_values = [
    0.2,
    0.3,
    0.4,
    0.5,
    0.6,
    0.7,
    0.8,
    0.9,
    1.0,
    1.3,
    1.5,
    1.8,
    2.0,
    2.5,
    3.0,
    4.0,
    5.0,
    6.0,
    8.0,
    10,
    12,
    15,
    20,
    30,
    40,
    50,
]

# reactions = ['T(d,n)4He', '3He(d,p)4He', 'D(d,p)T', 'D(d,n)3He']
reactions = ["T(d,n)4He", "D(d,p)T", "D(d,n)3He"]
# Note that Bosch and Hale show out of stated range reactivites for '3He(d,p)4He'
# so we do not show that column here.

for reaction in reactions:
    df = pd.DataFrame(sample_temperature_values, columns=["Temperature"])

for reaction in reactions:
    df[reaction] = df.apply(
        lambda row: format(
            reactivity.reactivity(
                T_i=row["Temperature"],
                reaction=reaction,
                method="parameterized",
                use_cache=False,
            ),
            "0.3e",
        ),
        axis=1,
    )
df

### Check integrated reactivites against parameterized reactivities

In [ ]:
log_temperature_values = np.logspace(math.log10(0.2), math.log10(10000), 100)
sigma_v_df = pd.DataFrame(log_temperature_values, columns=["Temperature"])

# reactions = ['T(d,n)4He', '3He(d,p)4He', 'D(d,p)T', 'D(d,n)3He']
reactions = ["T(d,n)4He", "D(d,p)T", "D(d,n)3He"]
methods = ["parameterized", "integrated"]

# for reaction in reactions:
df = pd.DataFrame(log_temperature_values, columns=["Temperature"])


def reactivity_wrapper(*args, **kwargs):
    try:
        r = reactivity.reactivity(*args, **kwargs)
    except (
        exceptions.ReactivityTemperatureTooHighError,
        exceptions.ReactivityTemperatureTooHighError,
    ):
        r = None
    return r


for reaction in reactions:
    for method in methods:
        df[reaction + " " + method] = df.apply(
            lambda row: reactivity_wrapper(
                T_i=row["Temperature"],
                reaction=reaction,
                method=method,
                use_cache=False,
            ),
            axis=1,
        )
df

In [ ]:
fig, ax = plt.subplots(dpi=300)
fig.set_size_inches(3, 3)
df.plot(
    x="Temperature",
    y=[
        "T(d,n)4He parameterized",
        "T(d,n)4He integrated",
        "D(d,p)T parameterized",
        "D(d,p)T integrated",
        "D(d,n)3He parameterized",
        "D(d,n)3He integrated",
    ],
    linewidth=1,
    style=["-", "--", "-", "--", "-", "--"],
    logy=True,
    logx=True,
    ax=ax,
)
# ax.set_xlim(0.2, 1000)
# ax.grid('on', which='both', axis='both')
# ax.set_ylim(1e-25, 1e-21)
# ax.set_ylim(0, 5e-22)
# ax.legend(reaction_legend_dict.values(), prop={'size': 6}, loc='lower right')

# ax.legend([legend_dict[l] for l in ax.get_legend_handles_labels()[1]])

# ax.set_xlabel(r'$T_{i} \; \si{(keV)}$')
# ax.set_ylabel(r'$\langle \sigma v \rangle \; \si{(\meter^{3} \second^{-1})}$ ')
fig.savefig(os.path.join("images", "reactivity_check.png"), bbox_inches="tight")

In [ ]:
fig, ax = plt.subplots(dpi=300)
fig.set_size_inches(3, 3)
df.plot(
    x="Temperature",
    y=["T(d,n)4He integrated"],
    linewidth=1,
    style=["-", "--", "-", "--"],
    logy=True,
    logx=True,
    ax=ax,
)
ax.set_ylim(1e-27, 5e-21)
ax.set_xlim(1, 1e4)

### Check pb11 integrated vs parameterized reactivities [not yet supported]

In [ ]:
log_temperature_values = np.logspace(math.log10(0.2), math.log10(500), 100)
sigma_v_df = pd.DataFrame(log_temperature_values, columns=["Temperature"])

legend_dict = {
    "integrated_11B(p,4He)4He4He": r"Integrated: $\mathrm{p+B^{11}} \rightarrow \mathrm{3 \alpha}$",
    "parameterized_11B(p,4He)4He4He": r"Parameterized: $\mathrm{p+B^{11}} \rightarrow \mathrm{3 \alpha}$",
}

# TODO figure out how to get p-11B reactivity at temperature > 500keV
for reaction in legend_dict.keys():
    sigma_v_df[reaction] = sigma_v_df.apply(
        lambda row: reactivity._reactivity_integrated(
            row["Temperature"],
            reaction=reaction.split("_")[1],
        ),
        axis=1,
    )
# sigma_v_df['sum DD'] = sigma_v_df[['D(d,p)T', 'D(d,n)3He']].sum(axis=1)
sigma_v_df